## Method Exploration
Here we explore the method suggested in `arXiv:1803.01485v3`. We obtain image representations by taking the output of the penultimate layer of a pre-trained model.  

In this notebook we use the [Caltech 101](http://www.vision.caltech.edu/Image_Datasets/Caltech101/) dataset and the `ResNet50` model taken from the Keras Applications

In [1]:
# Imports
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
import os
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

In [2]:
# Initialize Model
model = ResNet50(weights='imagenet', include_top=False)

In [3]:
# Run model for each picture and get representation
def get_representations(path, a=None, b=None):
    features_list = []
    for img_name in os.listdir(path)[a:b]:
        img_path = os.path.join(path, img_name)
        img = image.load_img(img_path,  target_size=(300,300))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        features = model.predict(x)
        features_list.append(features[0])
    return features_list

In [4]:
# Check classes
DATA_PATH = 'data'
print(os.listdir(DATA_PATH))

['pagoda', 'yin_yang', 'gerenuk', 'emu', 'binocular', 'pizza', 'dollar_bill', 'kangaroo', 'octopus', 'ibis', 'trilobite', 'cup', 'BACKGROUND_Google', 'stop_sign', 'stegosaurus', 'water_lilly', 'Faces_easy', 'flamingo_head', 'schooner', 'car_side', 'tick', 'ant', 'garfield', 'platypus', 'ketch', 'Faces', 'snoopy', 'lamp', 'headphone', 'dalmatian', 'wild_cat', 'ceiling_fan', 'umbrella', 'cellphone', 'stapler', 'pigeon', 'helicopter', 'camera', 'dragonfly', 'elephant', 'okapi', 'flamingo', 'Leopards', 'electric_guitar', 'crocodile_head', 'inline_skate', 'panda', 'scorpion', 'saxophone', 'watch', 'chandelier', 'cannon', 'crayfish', 'scissors', 'anchor', 'buddha', 'lotus', 'bonsai', 'strawberry', 'joshua_tree', 'mayfly', 'grand_piano', 'metronome', 'pyramid', 'accordion', 'bass', 'crab', 'hawksbill', 'cougar_body', 'mandolin', 'starfish', 'laptop', 'chair', 'rhino', 'euphonium', 'menorah', 'windsor_chair', 'butterfly', 'crocodile', 'beaver', 'wheelchair', 'brontosaurus', 'revolver', 'cougar

In [93]:
class1 = 'bonsai'
class2 = 'ewer'

In [94]:
# Get Representations by Class and split to train/test
class1_train = get_representations(os.path.join(DATA_PATH, class1), b=40)
class2_train = get_representations(os.path.join(DATA_PATH, class2), b=40)

class1_test = get_representations(os.path.join(DATA_PATH, class1), a=-25)
class2_test = get_representations(os.path.join(DATA_PATH, class2), a=-25)

In [95]:
# Make train X,y vectors from class representations
X_train = np.concatenate((class1_train, class2_train))
X_train = X_train.reshape(X_train.shape[0],-1)

# Binary Labels
class1_labels_tr = np.ones(len(class1_train))
class2_labels_tr = np.zeros(len(class2_train))
y_train = np.concatenate((class1_labels_tr, class2_labels_tr))

In [96]:
# Make test X,y vectors from class representations
X_test = np.concatenate((class1_test, class2_test))
X_test = X_test.reshape(X_test.shape[0],-1)

# Binary Labels
class1_labels_ts = np.ones(len(class1_test))
class2_labels_ts = np.zeros(len(class2_test))
y_test = np.concatenate((class1_labels_ts, class2_labels_ts))

In [97]:
# Train a C-Support Vector Classification from sklearn
clf = make_pipeline(StandardScaler(), SVC(kernel='sigmoid', gamma='auto'))
clf.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto', kernel='sigmoid'))])

In [98]:
# Predict
preds = clf.predict(X_test)

In [99]:
print("Accuracy: {}%".format(100*(preds == y_test).sum()/len(y_test)))

Accuracy: 98.0%
